# MPT-Calculator

## Software Overview:

The $\texttt{MPT-Calculator}$ is an open source python library to calculate the Magnetic Polarizability Tensor (MPT) using high order finite elements and the [NGSolve](https://ngsolve.org/) finite element package.  

**The MPT-Calulator has been updated and is now avaliable on a dedicated github page [https://github.com/MPT-Calculator/MPT-Calculator](https://github.com/MPT-Calculator/MPT-Calculator) under a GNU General Public License.**

## Requirements:

**For these examples, we are using Python 3.8**

$\texttt{MPT-Calculator}$ utilises $\texttt{NGSolve}$, and its associated meshing library $\texttt{Netgen}$ to perform the underlying FEM computations. Both libraries are avaliable via github ([https://github.com/NGSolve/](https://github.com/NGSolve/)) and their dedicated website [https://ngsolve.org/](https://ngsolve.org/) under an LPLG license.

In addition to the standard python libraries, $\texttt{MPT-Calculator}$ requires:
<ul>
  <li>Netgen</li>
  <li>NGSolve</li>
  <li>Numpy</li>
  <li>SciPy</li>
  <li>Matplotlib</li>
  <li>Mutltiprocessing_on_dill</li>
</ul>
which are all avaliable via <code>pip3</code>. For example entering

```python
>> pip3 install numpy
```
in the command line will install Numpy on MAC, Linux, and Windows machines


Finally, for parallel execution in Linux, we also need to set the following environment variables:
```bash
export OMPNUMTHREADS=1
export MKLNUMTHREADS=1
export MKLTHREADINGLAYER=sequential
```
in the bash.rc file.

## Structure:
Regarding the structure of the $\texttt{MPT-Calculator}$ library, the user is expected to interact with $\texttt{runner.py}$, $\texttt{Settings.py}$, and $\texttt{PlotterSettings.py}$ with $\texttt{PlotterSettings.py}$ rarely being needed.

The overall structure of the code is as follows:

```bash
.
├── Documentation
│   └── PythonCodeDocumentation.pdf
├── Functions
│   ├── add_truncated_region_to_mesh.py
│   ├── Checkvalid.py
│   ├── count_prismatic_elements.py
│   ├── FullSolvers.py
│   ├── MeshCreation.py
│   ├── ML_MPT_Predictor.py
│   ├── MPTFunctions.py
│   ├── MultiPermeability.py
│   ├── PlotEditor.py
│   ├── PlotEditorWithErrorBars.py
│   ├── Plotters.py
│   ├── PODFunctions.py
│   ├── PODPlotEditor.py
│   ├── PODPlotEditorWithErrorBars.py
│   ├── PODSolvers.py
│   ├── ResultsFunctions.py
│   ├── SingleSolve.py
│   └── step_to_vol_mesher.py
├── GeoFiles
│   ├── Claw_wodden_handle.geo
│   ├── Coin.geo
│   ├── Cylinder.geo
│   ├── DualBar.geo
│   ├── dualBox.geo
│   └── ...
├── OCC_Geometry
│   ├── OCC_bottle.py
│   ├── OCC_cylinder.py
│   ├── OCC_dualbar.py
│   ├── OCC_sphere.py
│   └── ...
├── Results
│   └── sphere
|       └── al_0.01_mu_1_sig_1e6
|           └── 1e1-1e5_40_el_57698_ord_2
|               ├── Data
|               |   └── ...
|               ├── Functions
|               |   └── ...
|               ├── Graphs
|               |   └── ...
|               ├── Input_files
|               |   └── ...
|               ├── PlotEditor.py
|               └── PlotterSettings.py
├── Results_2d
│   ├── PlotterSettings.py
├── Settings
│   ├── PlotterSettings.py
│   └── Settings.py
├── VolFiles
│   ├── Claw_wodden_handle.vol
│   ├── Knife_Cheap_Chef.vol
│   ├── OCC_cylinder.vol
│   ├── OCC_dualbar.vol
│   ├── OCC_sphere.vol
│   └── ...
├── Changelog for MPT-Calculator
├── __init__.py
├── LICENSE
├── main_2d.py
├── runner.py
├── README.md

```


## $\texttt{MPT-Calculator}$ Pipeline

The general pipeline for $\texttt{MPT-Calculator}$ is as follows:

 <img src="Figures/flowchart.png" alt="isolated"/>

## First Example:

First, we need to navigate to the project root directory. To check use:

In [ ]:
import os
os.getcwd()

and to set:

In [ ]:
root_dir = os.path.dirname(os.getcwd())
os.chdir(root_dir)
os.getcwd()

We now need to import the main function. $\texttt{runner.py}$ contains the starting point for MPT-Calculator. 

In [ ]:
from runner import main
from time import time

$\texttt{main}$ is a function taking variable input arguments for controlling the mesh size, order, geometry model, exciting frequencies, and alpha:
```python
main(h=2, order=2, curve_degree=5, start_stop=(), alpha='', geometry='default', frequency_array='default', use_OCC=False,
         use_POD=False, use_parallel=True)
```
These will be explained in detail later. For now, lets run the default settings.

In [ ]:
start_time = time();
main();
stop_time = time();

In [ ]:
print(f'Total time = {(stop_time - start_time)/60} minutes')

We have ran a simulation using the default values of MPT-Calculator. This uses a unit sphere, scaled by $\alpha=10^{-3}$, as the object, discretised using a coarse mesh and order $p=2$ elements.

This is repeated for each frequency over the range of 10 to $10^8$ rad/s in 40 logarithmically spaced samples.

### Saving Results

The output of the simulation, including any figures are saved in the 'Results' folder.

For example: 
```bash
MPT-Calculator/Results/sphere/al_0.001_mu_1_sig_1e6/1e1-1e8_40_el_9814_ord_2/
```

The results folder will contain a copy of the input files used for the simulation, including the mesh, .geo file, and a copy of the main function. This is so that the results can be recreated.

### Returning Values

Unlike the original MPT-Calculator, the new version allows the user to return variables from main.

```python
TensorArray, EigenValues, N0, Elements, Array, Ndofs, EddyCurrentTest = main()
```
 - $\texttt{TensorArray}$ is a complex Nx9 array containing each of the tensor coefficients for the 3x3 rank 2 MPT for each frequency of interest.
 - $\texttt{EigenValues}$ is a Nx3 array containing the eigenvalues for each frequency.
 - $\texttt{N0}$ is the 3x3 $\mathcal{N}^0$ coefficients.
 - $\texttt{Elements}$ is the integer number of elements in the mesh.
 - $\texttt{Array}$ is the Nx1 array containing the frequencies used for the sweep.
 - $\texttt{Ndofs}$ is a tuple containing the number of degrees of freedom used for the $\boldsymbol{\theta}^{(0)}$ and $\boldsymbol{\theta}^{(1)}$ finite element spaces.
 - $\texttt{EddyCurrentTest}$ will contain an estimated max frequency where the eddy current model holds if ```EddyCurentTest``` is set to True in ```Settings.py```. Otherwise it will return False.

This is to make it easier to integrate the $\texttt{MPT-Calculator}$ into other codebases.

In [ ]:
TensorArray, EigenValues, N0, Elements, Array, Ndofs, EddyCurrentTest = main()

```mermaid
graph TD here[this page] --> bookmarklet(use bookmarklet) bookmarklet --> make here --> pip(pip) here --> clone pip --> installed clone(clone) --> installed clone --> fork(fork) fork --> profit((profit)) make(make diagrams) --> profit installed --> make
```